# Lab 9: Úvod do autoencoderov

Na dnešnom cvičení sa oboznámime s novým typom neurónových sietí - autoencodermi. Autoencodery sú neurónové siete, ktoré slúžia na kompresiu údajov, avšak oproti tradičným metódam kompresie ich netreba vopred naprogramovať. Ďalšou ich vlastnosťou že sú špecificky určené na kompresiu údajov podobných ako dáta, na ktorých bola sieť natrénovaná. Podobne ako ďalšie metódy kompresie (napr. mp3, jpeg), aj autoencodery majú stratovú kompresiu.

Z topologického hľadiska sú autoencodery zvyčajne symetrické (síce nie je to nevyhnutné, v praxi takéto siete sa jednoduchšie trénujú). Cieľom autoencoderov je kopírovať vstup na výstupe tak, že skryté vrstvy majú menej neurónov ako vstup, resp. výstup. Autoencodery teda majú dve časti, prvá (encoder) má za úlohu skomprimovať údaje, kým druhá časť (decoder) reprodukuje vstup zo zakódovaného tvaru.

![](figures/lab9-autoencoder.png)

Dnes si vytvoríte dva jednoduché autoencodery pomocou knižnice Keras, zatiaľ bez konvolúcie. Kostru riešenia nájdete [tu](https://github.com/ianmagyar/dl-course/blob/master/labs/sources/lab9.py).

## 1. Načítanie potrebných knižníc

V riešení využijeme už známe knižnice `keras`, `numpy` a `matplotlib` pre vizualizáciu fungovania siete:

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np

## 2. Vizualizácia výsledkov

V kóde máte pripravenú funkciu `plot_results`, ktorá slúži na vizualizáciu fungovania siete. Pomocou knižnice `matplotlib` zobrazíme niekoľko ukážkových vstupov, ich zakódovaný tvar a rekonštruovaný obraz.

In [ ]:
def plot_results(x_test, encoded_imgs, decoded_imgs, n=10):
    plt.figure(figsize=(40, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(3, n, i + 1)
        plt.imshow(x_test[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display encoded
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(encoded_imgs[i].reshape(8, 4))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(3, n, i + 1 + n * 2)
        plt.imshow(decoded_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

## 3. Príprava údajov

Rovnako ako pri konvolučnej sieti, aj dnes použijeme dataset číslic MNIST, keďže ale nepoužijeme konvolúciu, tieto obrázky potrebujeme reprezentovať ako jedrozmerné vektory. Pred tým ale pixely normalizujeme, t.j. pretypujeme ich na numpyovský float a hodnoty od 0 po 255 namapujeme do intervalu 0 až 1.

Všimnite si, že z datasetu nenačítame labely (nahradíme ich znakom `_`). Dôvodom je, že autoencodery by mali mať na výstupe rovnakú hodnotu ako na vstupe, teda stačia nám vstupné údaje z datasetu.

In [ ]:
(mnist_train, _), (mnist_test, _) = mnist.load_data()

x_train = # TODO: normalize to range 0 to 1
x_test = # TODO: normalize to range 0 to 1
x_train = # TODO: reshape to a list of one-dimensional vectors
x_test = # TODO: reshape to a list of one-dimensional vectors

## 4. Shallow autoencoder

V ďalšom kroku zadefinujete najjednoduchší možní autoencoder: autoencoder s jednou skrytou vsrtvou.

### 4.1. Definícia vrstiev
Keďže vrstvy chceme využit niekoľkokrát, použijeme iný štýl definície ako ste na to zvyknutí. Namiesto toho, aby sme najprv vytvorili model a postupne sme do neho pridávali vrstvy teraz najprv zadefinujeme vrstvy, ktorých spojíme pomocou parametrov v tvare:

In [ ]:
dalsia_vrstva = Typ_vrstvy(parametre)(predosla_vrstva)

In [ ]:
input_img = # TODO: define input layer
encoded = # TODO: define hidden layer
decoded = # TODO: define output layer

### 4.2. Definícia modelov
Ak už máte vrstvy zadefinované, môžeme pristúpiť k definícii modelov. V tomto kroku zadefinujeme tri modely: celkový autoencoder, encoder, a decoder. Modely autoencodera a encodera viete zadefinovať na základe ich vstupu a výstupu v tvare:

In [ ]:
model_object = Model(vstupna_vrstva, vystupna_vrstva)

In [ ]:
autoencoder = # TODO: define autoencoder
encoder = # TODO: define encoder

Definícia modelu čistého decodera je menej priamočiara. Jeho vstupom je síce výstup encodera, ale neviem priamo použiť vrstvy z autoencodera, keďže tie sú navzájom prepojené. Namiesto toho vytvoríme novú vstupnú vrstvu a následne pridáme ďalšie potrebné vrstvy (v tomto prípade iba poslednú vrstvu autoencodera).

In [ ]:
encoded_input = # TODO: define input to decoder
decoder_layer = # TODO: define further layer of decoder
decoder = # TODO: define decoder

### 4.3. Trénovanie autoencodera

Neostáva nám nič iné, len natrénovať našu sieť. Pridajte chýbajúce parametre do volaní funkcií nasledovne:
* optimizer - napr. adadelta
* loss - napr. binary_crossentropy
* input - vstup autoencodera
* output - očakávaný výstup autoencodera
* epochs - napr. 25
* batch_size - napr. 256
* shuffle - nastavte na True pre náhodné poradie v trénovacej množine
* verbose - nastavte na 2 pre jednoduchší výpis progresu

In [ ]:
autoencoder.compile(# TODO: insert parameters: optimizer, loss
    )
autoencoder.fit(# TODO: insert parameters: input, output, epochs, batch_size, shuffle
    )

### 4.4. Testovanie autoencodera

Ak už máme autoencoder natrénovaný, môžeme vizualizovať jeho funkčnosť. Zavolajte funkciu `predict` nad testovacou množinou. Keďže chceme vizualizovať vstup, zakódovanú reprezentáciu, aj výstup autoencodera, funkciu potrebujeme zavolať aj pre encoder aj pre samotný autoencoder. Ak chcete testovať zvlášť aj decoder, upravte kód podľa potreby.

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = autoencoder.predict(x_test)

plot_results(x_test, encoded_imgs, decoded_imgs)

Zobrazia sa Vám tri riadky obrázkov, kde prvý riadok je vstup, druhý je zakódovaný tvar, a posledný riadok obsahuje rekonštruované obrázky, t.j. výstup autoencodera, napr.

![](figures/lab9-autoencoder-shallow.jpg)

## 5. Deep autoencoder

V ďalšom kroku rozšírime našu sieť pridávaním ďalších vrstiev (samotná topológia je na Vás, nižšie máte uvedený príklad). Odporúčame Vám použiť aktivačnú funkciu `relu` až na poslednú vrstvu, kde sa použije `sigmoid`.

Layer (type)                 Output Shape              Param #   
_________________________________________________________________
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_6 (Dense)              (None, 784)               101136    
_________________________________________________________________
Total params: 222,384
Trainable params: 222,384
Non-trainable params: 0

### 5.1. Definícia vrstiev

Zadefinujte jednotlivé vrstvy hlbokej neurónovej siete podobne ako v bode 4.1.

In [ ]:
input_img = # TODO: define input layer
encoded_1 = # TODO: define next layer; activation relu
encoded_2 = # TODO: define next layer; activation relu
encoded_3 = # TODO: define next layer; activation relu

decoded_1 = # TODO: define next layer; activation relu
decoded_2 = # TODO: define next layer; activation relu
decoded_3 = # TODO: define next layer; activation sigmoid

### 5. 2. Definícia modelov

Podobne ako v bode 4.2., zase zadefinujeme tri modely: celý autoencoder, iba encoder a iba decoder. Pri vytváraní modelu decodera dbajte na to, aby ste pridali všetky potrebné vrstvy.

In [ ]:
autoencoder = # TODO: define autoencoder

encoder = # TODO: define encoder

encoded_input = # TODO: define input to decoder
decoder_layer = # TODO: define next layer of the decoder
decoder_layer = # TODO: define next layer of the decoder
decoder_layer = # TODO: define next layer of the decoder
decoder = # TODO: define decoder

### 5. 3. Trénovanie autoencodera

V ďalšom kroku natrénujeme hlboký autoenoder rovnako ako v bode 4.3.

In [ ]:
autoencoder.compile(# TODO: insert parameters: optimizer, loss
    )
autoencoder.fit(# TODO: insert parameters: input, output, epochs, batch_size, shuffle
    )

### 5. 4. Testovanie autoencodera

Vizualizácia bude mať rovnaký tvar ako v bode 4.4. V tomto bode ale pre ukážku nepoužijeme autoencoder, ale zvlášť encoder a následne decoder.

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

plot_results(x_test, encoded_imgs, decoded_imgs)

![](figures/lab9-autoencoder-deep.jpg)